In [135]:
# Import Data Manipulation Library
import pandas as pd
import numpy as np

# import Data Visualisation Libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Warnings
import warnings
warnings.filterwarnings('ignore')

# Loggings
import logging
logging.basicConfig(level=logging.INFO,
                    filename="Model.log",
                    filemode="w",
                    format= "%(asctime)s - %(levelname)s - %(message)s", force=True)

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,learning_curve,KFold
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import xgboost
from xgboost import XGBRegressor
from sklearn.cluster import KMeans


# Multicolinearity test and treatment libraries
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA

In [136]:
pd.set_option("Display.max_columns", None)
pd.set_option("display.max_rows", 100)

In [137]:
url = "https://raw.githubusercontent.com/Digraskarpratik/SupplyChain_BusinessModel/refs/heads/main/SCM.csv"

df = pd.read_csv(url)

df.sample(frac=1)

,Ware_house_ID,WH_Manager_ID,Location_type,WH_capacity_size,zone,WH_regional_zone,num_refill_req_l3m,transport_issue_l1y,Competitor_in_mkt,retail_shop_num,wh_owner_type,distributor_num,flood_impacted,flood_proof,electric_supply,dist_from_hub,workers_num,wh_est_year,storage_issue_reported_l3m,temp_reg_mach,approved_wh_govt_certificate,wh_breakdown_l3m,govt_check_l3m,product_wg_ton
16201,WH_116201,EID_66201,Rural,Mid,East,Zone 4,3,0,8,3960,Company Owned,66,0,0,1,101,30.0,2008.0,18,0,B,3,26,20072
16460,WH_116460,EID_66460,Rural,Small,South,Zone 1,4,2,2,6627,Rented,34,0,0,1,148,31.0,2001.0,21,0,C,2,30,27120
16217,WH_116217,EID_66217,Rural,Mid,East,Zone 4,1,0,4,5676,Rented,43,0,0,1,171,34.0,NaN,9,0,C,2,26,11080
18978,WH_118978,EID_68978,Rural,Mid,South,Zone 2,3,1,4,5763,Rented,32,0,0,0,114,41.0,2015.0,11,1,B+,2,32,13083
857,WH_100857,EID_50857,Rural,Large,North,Zone 6,6,0,4,4989,Company Owned,57,0,0,0,239,25.0,2000.0,30,0,B,4,6,37083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6749,WH_106749,EID_56749,Urban,Mid,South,Zone 4,4,0,4,3908,Company Owned,49,1,1,1,247,35.0,2000.0,23,0,B+,4,26,26133
12738,WH_112738,EID_62738,Rural,Mid,West,Zone 3,2,0,4,3131,Rented,27,0,0,0,82,36.0,NaN,27,0,C,6,19,35124
7163,WH_107163,EID_57163,Rural,Large,North,Zone 6,3,1,3,5552,Company Owned,70,0,0,1,261,26.0,NaN,19,1,A+,6,23,25126
12108,WH_112108,EID_62108,Rural,Mid,North,Zone 2,6,3,2,5430,Company Owned,66,0,0,1,72,40.0,NaN,5,0,B,1,1,6119


In [138]:
df.isnull().sum()

Ware_house_ID                       0
WH_Manager_ID                       0
Location_type                       0
WH_capacity_size                    0
zone                                0
WH_regional_zone                    0
num_refill_req_l3m                  0
transport_issue_l1y                 0
Competitor_in_mkt                   0
retail_shop_num                     0
wh_owner_type                       0
distributor_num                     0
flood_impacted                      0
flood_proof                         0
electric_supply                     0
dist_from_hub                       0
workers_num                       877
wh_est_year                     10545
storage_issue_reported_l3m          0
temp_reg_mach                       0
approved_wh_govt_certificate      805
wh_breakdown_l3m                    0
govt_check_l3m                      0
product_wg_ton                      0
dtype: int64

In [139]:
df.drop(["wh_est_year", "workers_num", "approved_wh_govt_certificate", "Ware_house_ID", "WH_Manager_ID", "WH_regional_zone"], axis=1, inplace=True)

In [140]:
df.isnull().sum()

Location_type                 0
WH_capacity_size              0
zone                          0
num_refill_req_l3m            0
transport_issue_l1y           0
Competitor_in_mkt             0
retail_shop_num               0
wh_owner_type                 0
distributor_num               0
flood_impacted                0
flood_proof                   0
electric_supply               0
dist_from_hub                 0
storage_issue_reported_l3m    0
temp_reg_mach                 0
wh_breakdown_l3m              0
govt_check_l3m                0
product_wg_ton                0
dtype: int64

In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22150 entries, 0 to 22149
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Location_type               22150 non-null  object
 1   WH_capacity_size            22150 non-null  object
 2   zone                        22150 non-null  object
 3   num_refill_req_l3m          22150 non-null  int64 
 4   transport_issue_l1y         22150 non-null  int64 
 5   Competitor_in_mkt           22150 non-null  int64 
 6   retail_shop_num             22150 non-null  int64 
 7   wh_owner_type               22150 non-null  object
 8   distributor_num             22150 non-null  int64 
 9   flood_impacted              22150 non-null  int64 
 10  flood_proof                 22150 non-null  int64 
 11  electric_supply             22150 non-null  int64 
 12  dist_from_hub               22150 non-null  int64 
 13  storage_issue_reported_l3m  22150 non-null  in

In [142]:
from collections import OrderedDict

stats = []

for col in df.columns:
    if df[col].dtype != "object":
        numerical_stats = OrderedDict({
           'Feature': col,
            'Minimum': df[col].min(),
            'Maximum': df[col].max(),
            'Mean': df[col].mean(),
            "median" : df[col].median(),
            'Mode': df[col].mode()[0] if not df[col].mode().empty else None,
            '25%': df[col].quantile(0.25),
            '75%': df[col].quantile(0.75),
            'IQR': df[col].quantile(0.75) - df[col].quantile(0.25),
            'Standard Deviation': df[col].std(),
            'Skewness': df[col].skew(),
            'Kurtosis': df[col].kurt()
        })
        stats.append(numerical_stats)

# Convert to DataFrame
report = pd.DataFrame(stats)

# Outlier Identification :
outlier_label = []
for col in report['Feature']:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    LW = Q1 - 1.5 * IQR   # LW : Lower Whisker Line
    UW = Q3 + 1.5 * IQR   # UW : Upper Whisker Line
    outliers = df[(df[col] < LW) | (df[col] > UW)]
    if not outliers.empty:
        outlier_label.append("Has Outliers")
    else:
        outlier_label.append("No Outliers")

report["Outlier Comment"] = outlier_label

# Checking Report
report.style.background_gradient(subset= ["Minimum", "Maximum", "Mean", "median", "Mode", "25%", "75%", "IQR", "Standard Deviation", "Skewness", "Kurtosis"], cmap= "coolwarm")

,Feature,Minimum,Maximum,Mean,median,Mode,25%,75%,IQR,Standard Deviation,Skewness,Kurtosis,Outlier Comment
0,num_refill_req_l3m,0,8,4.097020,4.000000,5,2.000000,6.000000,4.000000,2.606289,-0.081390,-1.218193,No Outliers
1,transport_issue_l1y,0,5,0.777201,0.000000,0,0.000000,1.000000,1.000000,1.201747,1.605424,1.820683,Has Outliers
2,Competitor_in_mkt,0,12,3.103928,3.000000,2,2.000000,4.000000,2.000000,1.142886,0.985102,1.798511,Has Outliers
3,retail_shop_num,1821,11008,4983.115711,4859.000000,4808,4309.250000,5499.000000,1189.750000,1050.634225,0.905324,1.846915,Has Outliers
4,distributor_num,15,70,42.386998,42.000000,21,29.000000,56.000000,27.000000,16.057730,0.017210,-1.187118,No Outliers
5,flood_impacted,0,1,0.098691,0.000000,0,0.000000,0.000000,0.000000,0.298253,2.691308,5.243614,Has Outliers
6,flood_proof,0,1,0.054492,0.000000,0,0.000000,0.000000,0.000000,0.226991,3.925685,13.412215,Has Outliers
7,electric_supply,0,1,0.656072,1.000000,1,0.000000,1.000000,1.000000,0.475028,-0.657167,-1.568273,No Outliers
8,dist_from_hub,55,271,163.613725,164.000000,204,109.000000,218.000000,109.000000,62.660709,-0.009042,-1.200679,No Outliers
9,storage_issue_reported_l3m,0,39,17.116659,18.000000,24,10.000000,24.000000,14.000000,9.174193,0.117473,-0.678794,No Outliers


In [143]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22150 entries, 0 to 22149
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Location_type               22150 non-null  object
 1   WH_capacity_size            22150 non-null  object
 2   zone                        22150 non-null  object
 3   num_refill_req_l3m          22150 non-null  int64 
 4   transport_issue_l1y         22150 non-null  int64 
 5   Competitor_in_mkt           22150 non-null  int64 
 6   retail_shop_num             22150 non-null  int64 
 7   wh_owner_type               22150 non-null  object
 8   distributor_num             22150 non-null  int64 
 9   flood_impacted              22150 non-null  int64 
 10  flood_proof                 22150 non-null  int64 
 11  electric_supply             22150 non-null  int64 
 12  dist_from_hub               22150 non-null  int64 
 13  storage_issue_reported_l3m  22150 non-null  in

In [144]:
# Replace Outliers with Median Statergy

for col in df.select_dtypes(include='number').columns:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Identify outliers
    outliers = (df[col] < lower_bound) | (df[col] > upper_bound)
    outlier_count = outliers.sum()

    if outlier_count > 0:
        replacement = df[col].median()  
        df.loc[outliers, col] = replacement
        print(f"Replaced {outlier_count} outliers in '{col}' with median.")
    else:
        print(f"No outliers found in '{col}'.")

No outliers found in 'num_refill_req_l3m'.
Replaced 2628 outliers in 'transport_issue_l1y' with median.
Replaced 85 outliers in 'Competitor_in_mkt' with median.
Replaced 825 outliers in 'retail_shop_num' with median.
No outliers found in 'distributor_num'.
Replaced 2186 outliers in 'flood_impacted' with median.
Replaced 1207 outliers in 'flood_proof' with median.
No outliers found in 'electric_supply'.
No outliers found in 'dist_from_hub'.
No outliers found in 'storage_issue_reported_l3m'.
No outliers found in 'temp_reg_mach'.
No outliers found in 'wh_breakdown_l3m'.
No outliers found in 'govt_check_l3m'.
No outliers found in 'product_wg_ton'.


In [145]:
# Creating Our First Log
logging.info('Outlier Detection and Replacing of Outlier Activity Done Using IQR Method.')

In [146]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

df["Location_type"]  = le.fit_transform(df["Location_type"])
df["WH_capacity_size"] = le.fit_transform(df["WH_capacity_size"])
df["zone"] = le.fit_transform(df["zone"])
df["wh_owner_type"] = le.fit_transform(df["wh_owner_type"])


In [147]:
df.corr()["product_wg_ton"]

Location_type                 0.074158
WH_capacity_size             -0.002967
zone                         -0.008900
num_refill_req_l3m            0.000676
transport_issue_l1y          -0.100726
Competitor_in_mkt             0.006158
retail_shop_num              -0.005189
wh_owner_type                 0.003412
distributor_num               0.005143
flood_impacted                     NaN
flood_proof                        NaN
electric_supply              -0.000725
dist_from_hub                -0.006040
storage_issue_reported_l3m    0.986796
temp_reg_mach                 0.098452
wh_breakdown_l3m              0.343677
govt_check_l3m               -0.011977
product_wg_ton                1.000000
Name: product_wg_ton, dtype: float64

In [148]:
# Using PCA Concept:

# Step 1: Standardize the data

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df)

# Step 2: Determine number of components to retain 90% variance

for i in range(1, df.shape[1] + 1):
    pca = PCA(n_components=i)
    pca.fit(X_scaled)
    evr = np.cumsum(pca.explained_variance_ratio_)
    if evr[i - 1] >= 0.90:
        pcs = i
        break

print("Explained Variance Ratio:", evr)
print("Number of components selected:", pcs)

# Step 3: Apply PCA

pca = PCA(n_components=pcs)
pca_data = pca.fit_transform(X_scaled)

# Step 4: Create DataFrame

pca_columns = [f'PC{j+1}' for j in range(pcs)]
pca_df = pd.DataFrame(pca_data, columns=pca_columns)

# Step 5: Join Target Column with PCA:

pca_df = pca_df.join(df['product_wg_ton'], how = 'left')

pca_df

Explained Variance Ratio: [0.17406364 0.30899206 0.41702541 0.50320971 0.57914581 0.63939361
 0.695724   0.75045374 0.80126159 0.85115907 0.89517819 0.93378362]
Number of components selected: 12


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,product_wg_ton
0,0.197774,-0.320063,0.560026,0.682939,0.097685,0.432127,0.238833,-0.033639,-0.156185,-0.473081,0.864604,0.196973,17115
1,-0.589723,-0.486464,-0.011851,-0.398284,-0.251909,-0.196005,-0.285049,-0.359459,-0.007912,0.233023,0.008235,-0.236835,5074
2,0.053152,-0.356737,-0.802390,0.057717,0.363912,-0.090114,-0.115672,-0.194389,0.357732,0.169134,-0.102741,0.144877,23137
3,0.859645,0.736990,-0.143325,-0.036032,-0.138674,-0.149502,-0.402295,0.279542,0.236898,-0.170129,-0.076799,0.058405,22115
4,-0.571666,-0.293307,-0.009354,-0.484406,0.214658,0.427540,-0.088873,0.077872,0.144174,-0.200545,-0.166635,-0.047095,24071
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22145,0.221304,0.778611,0.630663,-0.286383,-0.212449,0.198643,0.256583,0.141651,-0.000345,0.170752,-0.087221,-0.625501,29138
22146,0.828773,-0.301751,-0.205126,0.653695,0.334611,0.362959,0.117135,-0.000804,0.303916,0.158732,-0.131453,-0.178484,33108
22147,0.062143,-0.146513,-0.799311,-0.350116,0.061560,0.724268,0.491355,0.378960,0.546801,-0.117210,-0.202426,-0.586308,24072
22148,0.834944,-0.285599,-0.163385,-0.530434,-0.417810,0.743295,0.087125,0.306082,-0.207001,0.166738,-0.123762,0.024695,15055


In [149]:
from sklearn.model_selection import train_test_split

X = pca_df.drop(["product_wg_ton"], axis=1)
y = pca_df["product_wg_ton"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


In [151]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score, r2_score
import xgboost
DC = DecisionTreeRegressor()
DC.fit(X_train, y_train)

y_pred_DC = DC.predict(X_test)

accuracy_score_DC = r2_score(y_test, y_pred_DC)

print(f'The R2 Score for Decision :- {round(accuracy_score_DC * 100)}%') 


The R2 Score for Decision :- 80%


In [152]:
RF = RandomForestRegressor()
RF.fit(X_train, y_train)

y_pred_RF = RF.predict(X_test)

r2_score_RF = r2_score(y_test, y_pred_RF)

print(f'The R2 Score for Random Forest :- {round(r2_score_RF * 100)}%') 

The R2 Score for Random Forest :- 91%
